In [1]:
import datetime as dt

date_s = dt.datetime(2024, 11, 16, 0, 0, 0, tzinfo=dt.UTC)
date_e = dt.datetime(2024, 11, 17, 0, 0, 0, tzinfo=dt.UTC)

In [2]:
import io
from urllib import parse, request

base_url = "https://mesonet.agron.iastate.edu/cgi-bin/request/gis/pireps.py?"
params = {
    "sts": date_s.isoformat().replace("+00:00", "Z"),
    "ets": date_e.isoformat().replace("+00:00", "Z"),
    "artcc": "_ALL",
    "fmt": "csv",
}
url = base_url + parse.urlencode(params)
url

'https://mesonet.agron.iastate.edu/cgi-bin/request/gis/pireps.py?sts=2024-11-16T00%3A00%3A00Z&ets=2024-11-17T00%3A00%3A00Z&artcc=_ALL&fmt=csv'

In [3]:
import pandas as pd

df = pd.read_csv(url)
df

,VALID,URGENT,AIRCRAFT,REPORT,ICING,TURBULENCE,ATRCC,LAT,LON,PRODUCT_ID
0,202411160000,F,B190,SAW UA /OV KSAW/TM 0000/FL036/TP B190/SK OVC03...,NaN,NaN,ZMP,202411160000-KMSC-UBUS01-PIREP,46.353640,-87.395360
1,202411160000,F,B738,CRG UA /OV CRG/TM 0000/FL370/TP B738/TB NEG,NaN,NEG,ZJX,202411160000-KMSC-UBUS01-PIREP,30.336100,-81.514700
2,202411160002,F,EA50,MTJ UA /OV MTJ045010/TM 0002/FL180/TP EA50/TB ...,NaN,MOD 200-180,ZDV,202411160000-KMSC-UBUS01-PIREP,38.622861,-107.746890
3,202411160002,F,PC12,PGA UA /OV TBC080040/TM 0002/FL200/TP PC12/TB ...,NaN,NEG,ZDV,202411160000-KMSC-UBUS01-PIREP,36.235775,-110.457169
4,202411160003,F,B738,ORD UA /OV CGT330020/TM 0003/FL060/TP B738/SK ...,NaN,SMOOTH,ZAU,202411160000-KMSC-UBUS01-PIREP,41.798699,-87.792584
...,...,...,...,...,...,...,...,...,...,...
2465,202411162357,F,C172,PLU UA /OV SEA160016 /TM 2357 /FL040 /TP C172 ...,NEG,NaN,ZSE,202411170000-KMSC-UBUS01-PIREP,47.194085,-122.179500
2466,202411162358,F,B77L,UA /OV 5430N11900W /TM 2358 /FL310 /TP B77L /T...,NaN,MOD,NaN,202411162359-CYEG-UACN10-PIREP,54.500000,-119.000000
2467,202411162358,F,A333,MKK UA /OV BAMBO/TM 2358/FL130/TP A333/SK TOPS...,NaN,NaN,ZLC,202411170000-KMSC-UBUS01-PIREP,40.599000,-116.874300
2468,202411162359,F,B738,CID UA /OV CID/TM 2359/FL300/TP B738/TB CONS L...,NaN,CONS LGT CHOP,ZAU,202411170000-KMSC-UBUS01-PIREP,41.882899,-91.724588


In [ ]:
df = df[["VALID", "REPORT"]]
df = df.rename(columns={"VALID": "Timestamp", "REPORT": "Report"})
df["Timestamp"] = pd.to_datetime(df["Timestamp"], format=f"%Y%m%d%H%M")
df

,Timestamp,Report
0,202411160000,SAW UA /OV KSAW/TM 0000/FL036/TP B190/SK OVC03...
1,202411160000,CRG UA /OV CRG/TM 0000/FL370/TP B738/TB NEG
2,202411160002,MTJ UA /OV MTJ045010/TM 0002/FL180/TP EA50/TB ...
3,202411160002,PGA UA /OV TBC080040/TM 0002/FL200/TP PC12/TB ...
4,202411160003,ORD UA /OV CGT330020/TM 0003/FL060/TP B738/SK ...
...,...,...
2465,202411162357,PLU UA /OV SEA160016 /TM 2357 /FL040 /TP C172 ...
2466,202411162358,UA /OV 5430N11900W /TM 2358 /FL310 /TP B77L /T...
2467,202411162358,MKK UA /OV BAMBO/TM 2358/FL130/TP A333/SK TOPS...
2468,202411162359,CID UA /OV CID/TM 2359/FL300/TP B738/TB CONS L...


In [ ]:
df[["ARTCC", "Location", "Time", "Flight Level", "Aircraft Type", "Rest"]] = df["Report"].str.split("/", n=5, expand=True)
df

,Timestamp,Report,ARTCC,Location,Time,Flight Level,Aircraft Type,Rest
0,2024-11-16 00:00:00,SAW UA /OV KSAW/TM 0000/FL036/TP B190/SK OVC03...,SAW UA,OV KSAW,TM 0000,FL036,TP B190,SK OVC036-TOPUNKN/RM DURD
1,2024-11-16 00:00:00,CRG UA /OV CRG/TM 0000/FL370/TP B738/TB NEG,CRG UA,OV CRG,TM 0000,FL370,TP B738,TB NEG
2,2024-11-16 00:02:00,MTJ UA /OV MTJ045010/TM 0002/FL180/TP EA50/TB ...,MTJ UA,OV MTJ045010,TM 0002,FL180,TP EA50,TB MOD 200-180/RM DURD ZDVWC-12
3,2024-11-16 00:02:00,PGA UA /OV TBC080040/TM 0002/FL200/TP PC12/TB ...,PGA UA,OV TBC080040,TM 0002,FL200,TP PC12,TB NEG/RM ZDVWC-36
4,2024-11-16 00:03:00,ORD UA /OV CGT330020/TM 0003/FL060/TP B738/SK ...,ORD UA,OV CGT330020,TM 0003,FL060,TP B738,SK THIN LAYER 040/TB SMOOTH
...,...,...,...,...,...,...,...,...
2465,2024-11-16 23:57:00,PLU UA /OV SEA160016 /TM 2357 /FL040 /TP C172 ...,PLU UA,OV SEA160016,TM 2357,FL040,TP C172,TA 02 /IC NEG
2466,2024-11-16 23:58:00,UA /OV 5430N11900W /TM 2358 /FL310 /TP B77L /T...,UA,OV 5430N11900W,TM 2358,FL310,TP B77L,TB MOD
2467,2024-11-16 23:58:00,MKK UA /OV BAMBO/TM 2358/FL130/TP A333/SK TOPS...,MKK UA,OV BAMBO,TM 2358,FL130,TP A333,SK TOPS 140
2468,2024-11-16 23:59:00,CID UA /OV CID/TM 2359/FL300/TP B738/TB CONS L...,CID UA,OV CID,TM 2359,FL300,TP B738,TB CONS LGT CHOP
